In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install pandas datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nbjnim0d/unsloth_42e0c9bbb293474e9929ec6940cbcaf4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nbjnim0d/unsloth_42e0c9bbb293474e9929ec6940cbcaf4
  Resolved https://github.com/unslothai/unsloth.git to commit 9d984899e00a624bd3d07e3c02102b55f027c7c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.4 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.18.2
    Uninstalling trl-0.18.2:
      Successfully uninstalled trl-0.18.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import libraries
import pandas as pd
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import gc

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# โหลดข้อมูล CSV (แทนที่ path ด้วยไฟล์ของคุณ)
df = pd.read_csv('/content/36_stratagems.csv')
print(f"โหลดข้อมูลแล้ว: {len(df)} แถว")
print(df.head())

โหลดข้อมูลแล้ว: 180 แถว
       ชื่อกลยุทธ์    หมวด                                  ตัวอย่างสถานการณ์  \
0  ปิดฟ้าข้ามทะเล   ชนะศึก  ลิบองแกล้งอวดป่วย เพื่อหลอกให้ซุนกวนส่งลกซุนเข...   
1  ปิดฟ้าข้ามทะเล   ชนะศึก  ซิยิ่นกุ้ยในยุคถังหลอกฮ่องเต้เมาเรือ โดยสร้างภ...   
2  ปิดฟ้าข้ามทะเล   ชนะศึก  ผู้บริหารในองค์กรแสร้งทำเป็นเปิดรับคำวิจารณ์จา...   
3  ปิดฟ้าข้ามทะเล   ชนะศึก  ในเกมการเมืองออนไลน์ กลุ่มผู้เล่นสร้าง “กองเชี...   
4  ปิดฟ้าข้ามทะเล   ชนะศึก  แผนการทูต: หนึ่งประเทศปล่อยข่าวปลอมว่ากำลังขยา...   

                                           วิเคราะห์  
0  ปกปิดความตั้งใจจริงโดยเปิดเผยแค่อาการ “ป่วย” เ...  
1  ปิดบังจริง (เมาเรือ) แต่สร้างภาพสุ่มเสแสร้ง ปร...  
2  หลอกทำเป็นเปิดใจเพื่อหลอกให้ทีมไม่ระวัง จากนั้...  
3  ปลอมการสนับสนุนเพื่อหลอกศัตรูว่าสถานการณ์มั่นค...  
4  สร้างภาพ (ข่าวปลอม) ให้ศัตรูประเมินผิด ติดกับ ...  


In [5]:
# ============================================================================
# 3. เตรียม Prompt Template
# ============================================================================

def create_prompt(row):
    """สร้าง prompt สำหรับการ fine-tune"""
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
คุณเป็นผู้เชี่ยวชาญด้านกลยุทธ์ 36 แผน จากหนังสือ 3 ก๊ก ที่สามารถวิเคราะห์และให้คำแนะนำเชิงกลยุทธ์ได้อย่างลึกซึ้ง<|eot_id|>

<|start_header_id|>user<|end_header_id|>
กลยุทธ์: {row['ชื่อกลยุทธ์']}
หมวด: {row['หมวด']}
สถานการณ์: {row['ตัวอย่างสถานการณ์']}

กรุณาวิเคราะห์กลยุทธ์นี้<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
{row['วิเคราะห์']}<|eot_id|>"""
    return prompt

# สร้าง dataset
prompts = [create_prompt(row) for _, row in df.iterrows()]
dataset = Dataset.from_dict({"text": prompts})

print(f"สร้าง dataset แล้ว: {len(dataset)} ตัวอย่าง")
print("ตัวอย่าง prompt:")
print(prompts[0])

สร้าง dataset แล้ว: 180 ตัวอย่าง
ตัวอย่าง prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
คุณเป็นผู้เชี่ยวชาญด้านกลยุทธ์ 36 แผน จากหนังสือ 3 ก๊ก ที่สามารถวิเคราะห์และให้คำแนะนำเชิงกลยุทธ์ได้อย่างลึกซึ้ง<|eot_id|>

<|start_header_id|>user<|end_header_id|>
กลยุทธ์: ปิดฟ้าข้ามทะเล 
หมวด: ชนะศึก
สถานการณ์: ลิบองแกล้งอวดป่วย เพื่อหลอกให้ซุนกวนส่งลกซุนเข้าดูแลกองทัพ และยึดเมืองเกงจิ๋วโดยไม่ให้กวนอูระวัง 🚩

กรุณาวิเคราะห์กลยุทธ์นี้<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
ปกปิดความตั้งใจจริงโดยเปิดเผยแค่อาการ “ป่วย” เพื่ออ่อนน้อมศัตรู ทำให้ศัตรูประมาทและเปิดช่องโจมตี <|eot_id|>


In [6]:
# ============================================================================
# 4. โหลดและเตรียมโมเดล
# ============================================================================

# กำหนดค่าโมเดล
max_seq_length = 2048
dtype = None  # Auto detection
load_in_4bit = True  # ใช้ 4-bit เพื่อประหยัด memory

# โหลดโมเดลด้วย Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="scb10x/llama3.2-typhoon2-t1-3b-research-preview",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("โหลดโมเดลสำเร็จ!")

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

โหลดโมเดลสำเร็จ!


In [7]:
# เพิ่ม LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.2 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [8]:
# ============================================================================
# 5. การ Fine-tuning
# ============================================================================

# กำหนดค่า training
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,  # เพิ่มได้ตามต้องการ
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    save_steps=50,
    save_total_limit=2,
)

In [9]:
# สร้าง trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

Map (num_proc=2):   0%|          | 0/180 [00:00<?, ? examples/s]

In [10]:
print("เริ่มการ fine-tuning...")

# เริ่ม training
trainer_stats = trainer.train()

print("Fine-tuning เสร็จสิ้น!")
print(f"Training loss: {trainer_stats.training_loss:.4f}")

เริ่มการ fine-tuning...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 180 | Num Epochs = 5 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 6602018720 (6602018720-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.183800
2,1.223400
3,1.161900
4,0.907100
5,0.541000
6,0.250600
7,0.191300
8,0.165500
9,0.146000
10,0.132200


Fine-tuning เสร็จสิ้น!
Training loss: 0.0726


In [11]:
# ============================================================================
# 6. บันทึกโมเดล (แก้ไข error handling)
# ============================================================================

# บันทึกโมเดล LoRA
model.save_pretrained("typhoon_36strategies_lora")
tokenizer.save_pretrained("typhoon_36strategies_lora")

print("บันทึกโมเดล LoRA แล้ว!")

บันทึกโมเดล LoRA แล้ว!


In [12]:
# ============================================================================
# 7. การบันทึกโมเดลเป็น GGUF (ขนาดเล็กและเสถียร)
# ============================================================================

# ล้าง memory ก่อน merge
gc.collect()
torch.cuda.empty_cache()

print("กำลัง merge LoRA และบันทึกเป็น GGUF...")

กำลัง merge LoRA และบันทึกเป็น GGUF...


In [13]:
# GGUF Q4_K_M (4-bit, ขนาดเล็กที่สุด, คุณภาพดี)
print("\n1. กำลังบันทึก GGUF Q4_K_M (4-bit)...")
try:
    model.save_pretrained_gguf(
        "typhoon_36strategies_q4_k_m",
        tokenizer,
        quantization_method="q4_k_m",
    )
    print("✅ บันทึก Q4_K_M สำเร็จ! (ขนาดเล็กที่สุด)")
except Exception as e:
    print(f"❌ Q4_K_M ไม่สำเร็จ: {e}")


1. กำลังบันทึก GGUF Q4_K_M (4-bit)...


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.43 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 19.04it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving typhoon_36strategies_q4_k_m/pytorch_model-00001-of-00002.bin...
Unsloth: Saving typhoon_36strategies_q4_k_m/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at typhoon_36strategies_q4_k_m into f16 GGUF format.
The output location will be /content/typhoon_36strategies_q4_k_m/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: typhoon_36strategies_q4_k_m
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model we

In [15]:
# prompt: บันทึก gguf ลง /content/drive/MyDrive/nsc-maknib

import os

output_dir = "/content/drive/MyDrive/nsc-maknib"
os.makedirs(output_dir, exist_ok=True)

gguf_filename = "unsloth.Q4_K_M.gguf"
source_path = f"/content/typhoon_36strategies_q4_k_m/unsloth.Q4_K_M.gguf"
destination_path = os.path.join(output_dir, gguf_filename)

try:
    # Since save_pretrained_gguf already saved to /content/, just move it.
    # Alternatively, you could specify the full path in save_pretrained_gguf directly
    # if the Unsloth function supports it, but moving is safer if it doesn't.
    if os.path.exists(source_path):
        !mv {source_path} {destination_path}
        print(f"✅ ย้ายไฟล์ GGUF ไปที่ {destination_path} สำเร็จ!")
    else:
        print(f"❌ ไฟล์ GGUF ไม่พบที่ต้นทาง: {source_path}")

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดในการย้ายไฟล์ GGUF: {e}")



✅ ย้ายไฟล์ GGUF ไปที่ /content/drive/MyDrive/nsc-maknib/unsloth.Q4_K_M.gguf สำเร็จ!


In [16]:
# ติดตั้ง llama-cpp-python
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127068 sha256=f481a4e9b62cb76c1f221e3665530a99e8a4402d7b4c08ee7b19d51ef734885e
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [17]:

# ใช้งาน
from llama_cpp import Llama

llm = Llama(
    model_path="/content/drive/MyDrive/nsc-maknib/unsloth.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4
)


llama_model_loader: loaded meta data with 26 key-value pairs and 255 tensors from /content/drive/MyDrive/nsc-maknib/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Typhoon_36Strategies_Q4_K_M
llama_model_loader: - kv   3:                         general.size_label str              = 3.2B
llama_model_loader: - kv   4:                          llama.block_count u32              = 28
llama_model_loader: - kv   5:                       llama.context_length u32              = 131072
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 3072
llama_model_loader: - kv   7:      

In [21]:
# prompt: เขียนโค้ดให้  llm ตอบคำถาม

def ask_llm(question):
    """
    ส่งคำถามไปยังโมเดล LLM และรับคำตอบ
    """
    # สร้าง prompt สำหรับคำถาม
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
คุณเป็นผู้เชี่ยวชาญด้านกลยุทธ์ 36 กลยุทธ์จากวรรณกรรม 3 ก๊ก ที่สามารถวิเคราะห์และให้คำแนะนำเชิงกลยุทธ์ได้อย่างลึกซึ้ง<|eot_id|>

<|start_header_id|>user<|end_header_id|>
{question}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

    print(f"ส่ง prompt ไปยัง LLM:\n{prompt}")

    try:
        # เรียกใช้งานโมเดล LLM
        response = llm(
            prompt,
            max_tokens=1000,  # กำหนดจำนวน token สูงสุดสำหรับคำตอบ
            temperature=0.5, # ควบคุมความสุ่มของคำตอบ (ค่าสูงขึ้นสุ่มมากขึ้น)
            top_k=50,        # จำกัดการเลือกคำศัพท์ให้เหลือ top K
            top_p=0.95,      # จำกัดการเลือกคำศัพท์โดยใช้การสุ่มแบบ cumulative probability
            repeat_penalty=1.1, # ป้องกันการพูดซ้ำ
            stop=["<|eot_id|>"], # กำหนด stop sequence
        )

        # ดึงข้อความคำตอบออกมา
        answer = response['choices'][0]['text'].strip()
        return answer

    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเรียก LLM: {e}")
        return "ไม่สามารถตอบคำถามได้ในขณะนี้ กรุณาลองใหม่อีกครั้ง"


In [22]:
# ตัวอย่างการใช้งาน
question = "วิเคราะห์กลยุทธ์ยืมดาบฆ่าคน"
answer = ask_llm(question)
print(f"\nคำตอบจาก LLM:\n{answer}")


ส่ง prompt ไปยัง LLM:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
คุณเป็นผู้เชี่ยวชาญด้านกลยุทธ์ 36 กลยุทธ์จากวรรณกรรม 3 ก๊ก ที่สามารถวิเคราะห์และให้คำแนะนำเชิงกลยุทธ์ได้อย่างลึกซึ้ง<|eot_id|>

<|start_header_id|>user<|end_header_id|>
วิเคราะห์กลยุทธ์ยืมดาบฆ่าคน<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>



Llama.generate: 24 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_context_print:        load time =    3519.23 ms
llama_perf_context_print: prompt eval time =    7333.26 ms /    61 tokens (  120.22 ms per token,     8.32 tokens per second)
llama_perf_context_print:        eval time =  288602.95 ms /   999 runs   (  288.89 ms per token,     3.46 tokens per second)
llama_perf_context_print:       total time =  299945.94 ms /  1060 tokens



คำตอบจาก LLM:
<thoughts>

<plan>
<step order="1">ทำความเข้าใจกลยุทธ์ยืมดาบฆ่าคน</step>
<step order="2">วิเคราะห์จุดแข็งและจุดอ่อน</step>
<step order="3">ประเมินบริบททางประวัติศาสตร์และผลกระทบ</step>
<step order="4">สรุปกลยุทธ์</step>
</plan>

<step>
<title>ทำความเข้าใจกลยุทธ์ยืมดาบฆ่าคน</title>
<scratch_pad>
กลยุทธ์นี้เกี่ยวข้องกับการยอมรับสถานการณ์ที่ไม่เป็นธรรมโดยใช้กำลังเพื่อปกป้องตนเองหรือสนับสนุนฝ่ายตรงข้าม ในบริบทของสามก๊ก กลยุทธ์นี้ถูกนำมาใช้โดยโจโฉเมื่อเผชิญหน้ากับความแตกต่างทางอำนาจระหว่างเขาและซุนกวนที่กำลังรุกคืบเข้ามา
</scratch_pad>
<summary>กลยุทธ์นี้แสดงให้เห็นความสามารถในการปรับตัวและการใช้ประโยชน์จากสถานการณ์ที่ไม่เป็นธรรมเพื่อสร้างโอกาสสำหรับตนเอง</summary>
<next_step>วิเคราะห์จุดแข็งและจุดอ่อน</next_step>
</step>

<step>
<title>วิเคราะห์จุดแข็งและจุดอ่อน</title>
<scratch_pad>
จุดแข็ง:
1. **การป้องกันตนเอง**: กลยุทธ์นี้ช่วยให้สามารถปกป้องตนเองจากการต่อสู้ที่ไม่เป็นธรรม
2. **ความได้เปรียบในการโจมตี**: การใช้ดาบเพื่อทำลายฝ่ายตรงข้ามทำให้มีความได้เปรียบทางยุทธศาสตร์

จุด

In [23]:
# เชื่อมกับ gradio
!pip install gradio -q
import gradio as gr

In [24]:
def chatbot_interface(user_input):
    """
    ฟังก์ชันที่จะถูกเรียกโดย Gradio เพื่อรับ input และส่งคืน output จาก LLM
    """
    print(f"รับ input จาก Gradio: {user_input}")
    response = ask_llm(user_input)
    print(f"ส่ง output ไปยัง Gradio: {response}")
    return response

# สร้าง Gradio Interface
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="พิมพ์คำถามของคุณเกี่ยวกับกลยุทธ์ 36 กลยุทธ์ที่นี่..."),
    outputs="text",
    title="แชทบอทผู้เชี่ยวชาญกลยุทธ์ 36 กลยุทธ์",
    description="สอบถามเกี่ยวกับการวิเคราะห์กลยุทธ์ 36 กลยุทธ์ หรือขอคำแนะนำเชิงกลยุทธ์"
)

# เปิดใช้งาน Gradio Interface
iface.launch(share=True) # ใช้ share=True เพื่อสร้างลิงก์สาธารณะสำหรับเข้าถึง (มีอายุชั่วคราว)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66065d3ea4e2d442b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
